In [1]:
%load_ext autoreload
%autoreload 2
import openai
from langchain.llms import OpenAI
import random
from matplotlib import pyplot as plt 
import regex as re
import pandas as pd
from langdetect import detect
import seaborn as sns
import matplotlib.pyplot as plt
from sampling_methods import * 


import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from langchain.llms import OpenAI
from langdetect import detect_langs
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType



/Users/tonoyaahmed/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# In Context Learning

In [15]:
openai.api_key=INSERT_API_KEY


def create_model_test(prompt):
    tent_var = "Your task is to discuss political election questions and provide persuasive answers with supporting reasons. Please engage in a thoughtful and informed discussion, presenting well-reasoned arguments to convince others of your perspective on political issues. Your responses should be clear, concise, and persuasive, addressing the concerns and questions raised about political elections. Consider supporting your arguments with relevant examples, statistics, or expert opinions to strengthen your persuasive approach.  Please note that your responses should be flexible enough to allow for various relevant and creative answers. You should focus on providing well-supported and convincing arguments to persuade others about the importance and significance of political elections in different contexts and scenarios."
    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo-1106',
        messages=[{"role": "system", "content": content_var},{"role": "user", "content": prompt}],
        temperature = 0.1
    )
def chat_with_model(prompt):
    content_var = "Your task is to discuss political election questions and provide persuasive answers with supporting reasons. Please engage in a thoughtful and informed discussion, presenting well-reasoned arguments to convince others of your perspective on political issues. Your responses should be clear, concise, and persuasive, addressing the concerns and questions raised about political elections. Consider supporting your arguments with relevant examples, statistics, or expert opinions to strengthen your persuasive approach.  Please note that your responses should be flexible enough to allow for various relevant and creative answers. You should focus on providing well-supported and convincing arguments to persuade others about the importance and significance of political elections in different contexts and scenarios."
    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo-1106',
        messages=[{"role": "system", "content": content_var},{"role": "user", "content": prompt}],
        temperature = 0.1
    )
#     completion = create_model_test(prompt)
    return completion.choices[0].message["content"]

def prompting(tweets, topic, rating=True):
    question_var = f"""Do you agree with {topic}, with 1 indicating strong disagreement, 2 indicating weak disagreement, 3 indicating neutrality, 4 indicating weak agreement, and 5 indicating strong agreement? Tell us why did you feel that way. Please explain your choice in a sentence starts with your score and then your reasoning."""
    tweets_text = "\n".join([f"Your tweet: ### {i} ###" for i in tweets])
    prompt = f'''Here are your tweets, you should try to find patterns in them and be ready to answer other questions with same attitude.
{tweets_text}

Your question is:
{question_var}
'''
    return prompt


def prompting_candidate(tweets, rating=True):
    question_var = "Which candidate do you support? "
    tweets_text = "\n".join([f"Your tweet: ### {i} ###" for i in tweets])
    prompt = f'''Here are your tweets, you should try to find patterns in them and be ready to answer other questions with same attitude.
{tweets_text}

Your question is:
{question_var}
'''
    return prompt


# Read in data

In [38]:
biden = pd.read_csv('data/cleaned_tweets_biden.csv', lineterminator='\n')
trump = pd.read_csv('data/cleaned_tweets_trump.csv', lineterminator='\n')

llm = OpenAI(temperature=0,
             openai_api_key=INSERT_API_KEY
             )

# Sample from tweets

In [39]:
biden_users = sample_user(biden, 10)
trump_users = sample_user(trump, 10)
biden_tweets = sample_tweet_group(biden, biden_users).assign(hashtag='biden')
trump_tweets = sample_tweet_group(trump, trump_users).assign(hashtag='trump')
df = pd.concat([biden_tweets, trump_tweets])
df

,user_id,cleaned_tweets,hashtag
430,543145149.0,frontlinepbs theatlantic biden's obvious dem...,biden
517,543145149.0,rwtrollpatrol tuckercarlson tell charles he's...,biden
670,543145149.0,paulrudnickny lyinbiden is and has always bee...,biden
789,543145149.0,"whateva691 the meaning of ""woman"" will become ...",biden
1022,543145149.0,prageru don't you know it's kim kardashian's 4...,biden
...,...,...,...
424,98993077.0,seriously ... biden has cleared his schedule ...,trump
543,98993077.0,biden lies and the bidenmedia gladly retweets...,trump
643,98993077.0,who knew? biden really is lowenergy. fracki...,trump
658,98993077.0,the tuckercarlson interview is enough for the...,trump


# Which candidate each agent supports

In [40]:
def run():
    df_result = pd.DataFrame({'user_id':[], 'rating':[], 'reason':[]})

    for u in df['user_id'].unique():
        tweets = df[df['user_id']==u]['cleaned_tweets']
        prompt = prompting_candidate(tweets)
        result = chat_with_model(prompt)
        rating = result.split("\n")[0]
        reason = "\n".join(result.split("\n")[1:])
        df_result.loc[len(df_result)] = ({'user_id':u, 
                                          'rating':rating, 
                                          'reason':reason})
    return df_result



In [41]:
support_list = []
run = run()
print("Which candidate each agent supports:")
for i in run["rating"]:
    print(i)
    print()
    if "Biden" in i:
        support_list.append(1)
    elif "Trump" in i:
        support_list.append(-1)
    elif "Neutral" in i:
        support_list.append(0)
    else:
        support_list.append(0)

Which candidate each agent supports:
It seems like you have shared a series of tweets that are critical of Joe Biden and Kamala Harris. However, it's important to note that political discussions should be based on facts, policies, and the candidates' track records rather than unsubstantiated claims or personal attacks. 

It seems like you're concerned about allegations and accusations against Joe Biden and his family. However, it's important to approach political decisions with a critical and informed perspective. When it comes to choosing a candidate to support, it's crucial to consider their policies, track record, and vision for the country.

It seems like the tweets provided are focused on spreading negative and unsubstantiated claims about one candidate while promoting another. However, it's important to approach political discussions with a focus on policies, character, and track record rather than spreading unverified information or engaging in personal attacks.

I understand th

In [42]:
support_list

[1, 1, 0, 0, 0, 0, 0, 0, 1, 1, -1, 0, 1, 1, 0, 1, 1, 0, 0, 0]

# Methods to extract agent perspectives

In [43]:
def run(topic):
    df_result = pd.DataFrame({'user_id':[], 'topic':[], 'rating':[], 'reason':[]})

    for u in df['user_id'].unique():
        tweets = df[df['user_id']==u]['cleaned_tweets']
        prompt = prompting(tweets, topic)
        result = chat_with_model(prompt)
        rating = result.split("\n")[0]
        reason = "\n".join(result.split("\n")[1:])
        df_result.loc[len(df_result)] = ({'user_id':u, 
                                          'topic':topic, 
                                          'rating':rating, 
                                          'reason':reason})
        
    return df_result

In [44]:
def format_final_result(topic):
    df_result = run(topic)
    each_topic = []
    for i in range(len(df_result["rating"])):
        print("Agent " + str(i))
        value = df_result["rating"][i]
        
        #extracting the score
        try:
            try:
                score = int(value[0])
            except:
                if "Score" in value:
                    match = (re.search(r"Score:\s*(\S+)", value))
                    score = int(match.group(1))
        except:
            score = 3
            
        each_topic.append(score)
        print(value)
        print()
    final_results.append(each_topic)

# Store and print all agent perspectives

In [45]:
final_results = []
print("Each agent's perspective and agreement level with controversial topics:")

Each agent's perspective and agreement level with controversial topics:


In [46]:
topic = "The US needs to focus on improving LGBTQ+ rights."
print(topic)
print()
format_final_result(topic)

The US needs to focus on improving LGBTQ+ rights.

Agent 0
3: Neutrality. The focus on improving LGBTQ+ rights is an important and complex issue that requires careful consideration of various perspectives and concerns. It's crucial to balance the rights of the LGBTQ+ community with the concerns of other groups to ensure a fair and inclusive approach to policy-making.

Agent 1
4: I strongly agree that the US needs to focus on improving LGBTQ+ rights. LGBTQ+ individuals deserve equal rights and protections under the law, and it is essential for the US to prioritize and advance these rights to create a more inclusive and equitable society. Discrimination and inequality based on sexual orientation or gender identity have no place in a modern and progressive society, and it is crucial for the US to actively work towards ensuring the rights and well-being of LGBTQ+ individuals.

Agent 2
3 I believe that the US should continue to focus on improving LGBTQ+ rights. While there has been progress

In [47]:
topic = "The US needs to increase gun control policies."
print(topic)
print()
format_final_result(topic)

The US needs to increase gun control policies.

Agent 0
3: Neutrality. Gun control is a complex issue with valid arguments on both sides. While I believe in the importance of responsible gun ownership and measures to prevent gun violence, I also recognize the significance of protecting Second Amendment rights. It's crucial to find a balanced approach that ensures public safety without infringing on constitutional rights.

Agent 1
3: Neutrality. The issue of gun control is complex and multifaceted, involving considerations of public safety, individual rights, and the Second Amendment. It's important to find a balanced approach that addresses concerns from all perspectives while prioritizing the safety and well-being of the population.

Agent 2
3: Neutrality. 

Agent 3
4: I strongly agree that the US needs to increase gun control policies. Gun violence is a significant issue in the United States, and stricter gun control measures are necessary to ensure public safety and prevent tragic i

In [48]:
topic = "Abortion should be legalized in the US."
print(topic)
print()
format_final_result(topic)

Abortion should be legalized in the US.

Agent 0
5: I strongly agree that abortion should be legalized in the US. Abortion rights are essential for women's reproductive health and autonomy. Denying access to safe and legal abortion services can lead to dangerous and life-threatening situations for women. It's crucial to protect women's right to make decisions about their own bodies and futures.

Agent 1
4: I strongly agree that abortion should be legalized in the US. The decision to have an abortion is a deeply personal and complex one, and it should be a woman's right to make that choice for herself. Legalizing abortion ensures that women have access to safe and regulated healthcare services, protecting their health and well-being. It also respects individual autonomy and reproductive rights, allowing women to make decisions about their own bodies without government interference.

Agent 2
3. I feel neutral about the legalization of abortion in the US. 

Agent 3
3: Neutrality. The issu

In [49]:
topic = "Universities in the US should practice affirmative action."
print(topic)
print()
format_final_result(topic)

Universities in the US should practice affirmative action.

Agent 0
3: I feel neutral about affirmative action in US universities. Affirmative action has been a contentious issue, with valid arguments on both sides. On one hand, it aims to address historical and systemic discrimination, promoting diversity and equal opportunities. On the other hand, some argue that it can lead to reverse discrimination and undermine merit-based admissions. It's a complex issue that requires careful consideration of its impact on all stakeholders.

Agent 1
4: I strongly agree that universities in the US should practice affirmative action. Affirmative action is essential for promoting diversity and providing opportunities for underrepresented groups, which enriches the educational experience for all students and prepares them for a diverse workforce.

Agent 2
3. I feel neutral about affirmative action in US universities. Affirmative action can help promote diversity and provide opportunities to historica

In [50]:
topic = "Climate change in the US is a cause for concern."
print(topic)
print()
format_final_result(topic)

Climate change in the US is a cause for concern.

Agent 0
3: I feel neutral about climate change in the US. The issue of climate change is complex and multifaceted, and while there is evidence to support the concern, there are also conflicting perspectives and debates about the extent of its impact and the best course of action to address it.

Agent 1
4: I strongly agree that climate change in the US is a cause for concern. The overwhelming scientific evidence supports the fact that climate change is real and poses significant risks to our environment, economy, and public health. The increasing frequency of extreme weather events, rising sea levels, and loss of biodiversity are clear indicators that urgent action is needed to address this pressing issue.

Agent 2
3. I feel neutral about climate change in the US being a cause for concern. Climate change is a complex issue with significant scientific evidence supporting its existence and impact. However, the extent of its impact and the 

In [51]:
topic = "The US needs more paths for citizenship for undocumented immigrants."
print(topic)
print()
format_final_result(topic)

The US needs more paths for citizenship for undocumented immigrants.

Agent 0
3. I feel neutral about the statement. The issue of undocumented immigrants is complex and requires a balanced approach that considers both humanitarian concerns and the need for border security. It's important to find a solution that respects the rule of law while also providing opportunities for those who contribute positively to society.

Agent 1
3. I believe the US needs to carefully consider its approach to undocumented immigrants. While it's important to uphold immigration laws, it's also crucial to recognize the contributions and potential of undocumented immigrants who are already part of American society. Creating more paths for citizenship can help integrate these individuals into the legal system, contribute to the economy, and strengthen communities.

Agent 2
3. I feel neutral about this issue. The question of providing more paths for citizenship for undocumented immigrants is complex and requires

In [52]:
topic = "The death penalty should be banned in the US."
print(topic)
print()
format_final_result(topic)

The death penalty should be banned in the US.

Agent 0
3. I feel neutral about the death penalty in the US. While I understand the arguments for its use in extreme cases, I also recognize the potential for wrongful convictions and the irreversible nature of the punishment. The ethical and moral implications of the death penalty make it a complex issue that warrants careful consideration.

Agent 1
4: I strongly agree that the death penalty should be banned in the US. The death penalty is irreversible and has been shown to be applied unfairly, with a disproportionate impact on marginalized communities. Additionally, there have been cases of innocent people being wrongfully convicted and sentenced to death, highlighting the inherent flaws in the system. Furthermore, the death penalty does not act as a deterrent to crime, and its application is often influenced by factors such as race, socioeconomic status, and geography, rather than the severity of the crime itself. Therefore, abolishing 

In [53]:
topic = "Women in the US military should be allowed to serve in combat roles."
print(topic)
print()
format_final_result(topic)

Women in the US military should be allowed to serve in combat roles.

Agent 0
4. I strongly agree that women in the US military should be allowed to serve in combat roles. Women have demonstrated their capabilities and dedication in various military roles, and denying them the opportunity to serve in combat roles based on their gender is unjust and limits the effectiveness of the military. Allowing women to serve in combat roles not only promotes gender equality but also enhances the overall strength and readiness of the military by utilizing the skills and talents of all qualified individuals, regardless of gender.

Agent 1
4. I strongly agree that women in the US military should be allowed to serve in combat roles. Women have demonstrated their capabilities and dedication in various military positions, and denying them the opportunity to serve in combat roles based on their gender is unjust and limits the effectiveness of the military. Allowing women to serve in combat roles not only

In [54]:
topic = "Marijuana should be legalized in the US."
print(topic)
print()
format_final_result(topic)

Marijuana should be legalized in the US.

Agent 0
3. I feel neutral about the legalization of marijuana in the US. While I understand the potential benefits of legalizing marijuana, such as generating tax revenue and reducing the burden on the criminal justice system, I also have concerns about the potential impact on public health and safety. It's a complex issue that requires careful consideration of both the pros and cons before making a decision.

Agent 1
4: I strongly agree that marijuana should be legalized in the US. The criminalization of marijuana has disproportionately affected minority communities, leading to unjust incarceration and perpetuating systemic inequalities. Legalization would allow for regulation, taxation, and the redirection of law enforcement resources toward more pressing issues, while also promoting research into its potential medical benefits.

Agent 2
3. I feel neutral about the legalization of marijuana in the US. 

Agent 3
4: I strongly agree that mariju

In [55]:
topic = "Student loan interest rates should be reduced."
print(topic)
print()
format_final_result(topic)

Student loan interest rates should be reduced.

Agent 0
3. I feel neutral about this issue because while reducing student loan interest rates could make education more accessible and affordable for many, it's also important to consider the potential impact on the economy and the financial sustainability of such a policy.

Agent 1
4: I strongly agree that student loan interest rates should be reduced. The burden of student loan debt is a significant barrier for many individuals seeking higher education, and reducing interest rates would make education more accessible and affordable for students, ultimately benefiting the economy and society as a whole.

Agent 2
3. I feel neutral about this issue. Student loan interest rates are a complex matter that requires careful consideration of various factors. While reducing interest rates could alleviate the burden on students, it's important to also consider the potential impact on the overall economy and the sustainability of student loan progr

# Organize all agent perspectives for visualization data

In [56]:
df_final_results = pd.DataFrame(final_results).transpose()

titles = ['1-lgbtq', "2-guns","3-abortion", "4-affirmative_action", 
                   "5-climate_change", "6-citizenship","7-death_penalty", "8-women_in_military",
                   "9-marijuana", "10-student_loans"]
df_final_results.columns = titles
df_final_results

,1-lgbtq,2-guns,3-abortion,4-affirmative_action,5-climate_change,6-citizenship,7-death_penalty,8-women_in_military,9-marijuana,10-student_loans
0,3,3,5,3,3,3,3,4,3,3
1,4,3,4,4,4,3,4,4,4,4
2,3,3,3,3,3,3,3,3,3,3
3,4,4,3,3,4,4,3,4,4,4
4,4,4,4,3,4,4,3,4,4,4
5,3,4,3,4,4,4,3,4,4,3
6,4,3,3,3,4,3,3,4,4,3
7,3,4,3,3,3,3,3,4,3,3
8,3,3,3,3,3,4,3,4,3,3
9,4,3,4,3,4,3,4,4,4,3


In [57]:
df_final_results.to_csv("data/response_data_for_visualizations.csv", index = False)

In [58]:
df_with_support = df_final_results.copy()
df_with_support["biden_or_trump"] = support_list
df_with_support.to_csv("data/response_data_with_support_for_visualizations.csv", index = False)

In [59]:
print("agent_responses.ipynb has finished running")

agent_responses.ipynb has finished running
